<a href="https://colab.research.google.com/github/ssawant/llm-forge/blob/main/sft_models/Mearge_LLMs_with_mergekit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merge LLMs with mergekit

Run this notebook on Google Colab with `T4 GPU (VRAM offloading)`


### TIES-Merging

```
models:
  - model: mistralai/Mistral-7B-v0.1
    # no parameters necessary for base model
  - model: OpenPipe/mistral-ft-optimized-1218
    parameters:
      density: 0.5
      weight: 0.5
  - model: santoshsawant/NeuralHermes-2.5-Mistral-7B
    parameters:
      density: 0.5
      weight: 0.3
merge_method: ties
base_model: mistralai/Mistral-7B-v0.1
parameters:
  normalize: true
dtype: float16
```

### SLERP

```
slices:
  - sources:
      - model: OpenPipe/mistral-ft-optimized-1218
        layer_range: [0, 32]
      - model: santoshsawant/NeuralHermes-2.5-Mistral-7B
        layer_range: [0, 32]
merge_method: slerp
base_model: OpenPipe/mistral-ft-optimized-1218
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16
```

### Passthrough
```
slices:
  - sources:
    - model: OpenPipe/mistral-ft-optimized-1218
      layer_range: [0, 32]
  - sources:
    - model: santoshsawant/NeuralHermes-2.5-Mistral-7B
      layer_range: [24, 32]
merge_method: passthrough
dtype: bfloat16
```

In [4]:
!git clone https://github.com/cg123/mergekit.git
!cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 1136, done.
remote: Counting objects: 100% (414/414), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 1136 (delta 347), reused 282 (delta 270), pack-reused 722
Receiving objects: 100% (1136/1136), 281.12 KiB | 23.43 MiB/s, done.
Resolving deltas: 100% (796/796), done.
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for mergekit (pyproject.toml) ... done


In [9]:
import yaml

MODEL_NAME = "NeuralHermes-7B-slerp"
yaml_config = """
slices:
  - sources:
      - model: santoshsawant/NeuralHermes-2.5-Mistral-7B
        layer_range: [0, 32]
      - model: EmbeddedLLM/Mistral-7B-Merge-14-v0.1
        layer_range: [0, 32]
merge_method: slerp
base_model: santoshsawant/NeuralHermes-2.5-Mistral-7B
parameters:
  t:
    - filter: self_attn
      value: [0, 0.5, 0.3, 0.7, 1]
    - filter: mlp
      value: [1, 0.5, 0.7, 0.3, 0]
    - value: 0.5
dtype: bfloat16

"""

# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

## Merge models

In [10]:
!mergekit-yaml config.yaml merge --copy-tokenizer --allow-crimes --out-shard-size 1B --lazy-unpickle

Fetching 8 files: 100% 8/8 [00:00<00:00, 18275.83it/s]
Fetching 11 files: 100% 11/11 [00:00<00:00, 21670.90it/s]
  0% 0/291 [00:00<?, ?it/s]WARNING:root:Using common submatrix of size torch.Size([32000, 4096]) for model.embed_tokens.weight
 70% 203/291 [02:58<00:44,  1.98it/s]WARNING:root:Using common submatrix of size torch.Size([32000, 4096]) for lm_head.weight
100% 291/291 [04:22<00:00,  1.11it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
!pip install -qU huggingface_hub

from huggingface_hub import ModelCard, ModelCardData
from jinja2 import Template

username = "santoshsawant"

template_text = """
---
license: apache-2.0
tags:
- merge
- mergekit
- lazymergekit
{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a merge of the following models using [mergekit](https://github.com/cg123/mergekit):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}

## Configuration

```yaml
{{- yaml_config -}}
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Get list of models from config
data = yaml.safe_load(yaml_config)
if "models" in data:
    models = [data["models"][i]["model"] for i in range(len(data["models"])) if "parameters" in data["models"][i]]
elif "parameters" in data:
    models = [data["slices"][0]["sources"][i]["model"] for i in range(len(data["slices"][0]["sources"]))]
elif "slices" in data:
    models = [data["slices"][i]["sources"][0]["model"] for i in range(len(data["slices"]))]
else:
    raise Exception("No models or slices found in yaml config")

# Fill the template
content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
)

# Save the model card
card = ModelCard(content)
card.save('merge/README.md')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 4.6 MB/s eta 0:00:00


In [13]:
from google.colab import userdata
from huggingface_hub import HfApi

username = "santoshsawant"

# Defined in the secrets tab in Google Colab
api = HfApi(token=userdata.get("huggingface"))

api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model"
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path="merge",
)

Upload 9 LFS files:   0%|          | 0/9 [00:00<?, ?it/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.99G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/1.94G [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/755M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/santoshsawant/NeuralHermes-7B-slerp/commit/57dae104809557372cabe688eb608448d32fa485', commit_message='Upload folder using huggingface_hub', commit_description='', oid='57dae104809557372cabe688eb608448d32fa485', pr_url=None, pr_revision=None, pr_num=None)